In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import sys
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import scipy.io
import time
drive_path = '/Volumes/Brain2016'
manifest_path = os.path.join(drive_path,'BrainObservatory/manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_path)
good=np.load('good_dfof3_cell_ids.npy')


In [11]:
# get one celltype to pick data
#L23_VISp=boc.get_ophys_experiments(targeted_structures=['VISp'], imaging_depths=[175], cre_lines=['Cux2-CreERT2'],stimuli=['locally_sparse_noise'])

L4_Cux2_VISp=boc.get_ophys_experiments(targeted_structures=['VISp'], imaging_depths=[275], cre_lines=['Cux2-CreERT2'],stimuli=['locally_sparse_noise'])
#L4_Rorb_VISp=boc.get_ophys_experiments(targeted_structures=['VISp'],  cre_lines=['Rorb-IRES2-Cre'],stimuli=['locally_sparse_noise'])
#L4_Scnn1a_VISp=boc.get_ophys_experiments(targeted_structures=['VISp'],  cre_lines=['Scnn1a-Tg3-Cre'],stimuli=['locally_sparse_noise'])

cell_type='L4_Cux2_VISp'

In [12]:
# stim_table = data_set_c.get_stimulus_table()
# lsn = data_set_c.get_stimulus_template('locally_sparse_noise')
# plt.imshow(lsn[0,:,:], cmap='gray', interpolation='none')

In [13]:
i=0
matlabdict={}
for experiment in eval(cell_type):
    data=boc.get_ophys_experiment_data(ophys_experiment_id = experiment['id'])
#select good cells with dfof >3%
    cells=data.get_cell_specimen_ids()
    good_cells=[n for n in cells if n in good]  
    timestamps,exp_dff=data.get_dff_traces(cell_specimen_ids=good_cells)
    running_speed,_ = data.get_running_speed()
### this is the line to modify for different stimulus
    stim_time=data.get_stimulus_table('locally_sparse_noise')
    section=len(stim_time)-1
    timestamps_all=timestamps[stim_time['start'][0]:stim_time['end'][section]]
    exp_dfof_all=exp_dff[:,stim_time['start'][0]:stim_time['end'][section]]
    running_speed_all=running_speed[stim_time['start'][0]:stim_time['end'][section]]
    matlabdict[cell_type+str(i)]=exp_dfof_all
    matlabdict['running_speed'+str(i)]= running_speed_all
    matlabdict['timestamps_'+str(i)]= timestamps_all
    i=i+1
    

In [14]:
savingfile='/Users/Jensun/Dropbox/SWDB-2016/project/localsparsenoise/'+cell_type
scipy.io.savemat(savingfile,matlabdict)